### Transformer

In [1]:
from datasets import load_dataset
from transformers import BertModel
from transformers import pipeline
import pandas as pd
import numpy as np

/Users/linghuang/miniconda3/envs/llm-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Search semantics

The Hugging Face summarization task page lists models that support summarization. In this section, we will the following resources:

- data: We will work with the 'xsum dataset,' containing a collection of BBC articles and their corresponding summaries. This dataset serves as the foundation for our tasks.

- model: Our chosen model is the 't5-small model,' with 60 million parameters (equivalent to 242MB for PyTorch). T5, an encoder-decoder model developed by Google, boasts versatility, supporting various tasks including summarization, translation, question-answering, and text classification.

In [2]:
xsum_dataset = load_dataset(
    "xsum", version="1.2.0"
)  

/Users/linghuang/miniconda3/envs/llm-rag/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


This dataset provides 3 columns:

- document: the BBC article text
- summary: a "ground-truth" summary --> Note how subjective this "ground-truth" is. Is this the same summary you would write? This a great example of how many LLM applications do not have obvious "right" answers.
- id: article ID

In [8]:
# Taking a sample of 100 rows
xsum_sample = xsum_dataset["train"].select(range(1000)).to_pandas()

# Combining 'document' and 'summary' columns
xsum_sample["combined"] = (
    "Document: " + xsum_sample.document.str.strip() + "; Summary: " + xsum_sample.summary.str.strip()
)

In [9]:
from sentence_transformers import SentenceTransformer

#encoding the data
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
encoded_data = encoder.encode(xsum_sample["combined"])

In [10]:
encoded_data

array([[-0.12973613, -0.07995621, -0.02103525, ...,  0.01458147,
        -0.04181118,  0.05969834],
       [-0.10183043, -0.00813398,  0.01535375, ...,  0.03995895,
        -0.10245819,  0.08624592],
       [-0.06544365, -0.22466174,  0.01042669, ...,  0.06865789,
         0.0731439 ,  0.01244215],
       ...,
       [-0.00177471, -0.11989915,  0.03554984, ...,  0.01920111,
        -0.06250598, -0.0915705 ],
       [-0.15317537,  0.04223507, -0.00396573, ...,  0.15491395,
        -0.01295559, -0.09967731],
       [-0.11358325,  0.03419029,  0.06424171, ...,  0.04367112,
        -0.21225762,  0.05421832]], dtype=float32)

In [11]:
import faiss

In [12]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

In [ ]:
faiss.normalize_L2(encoded_data)
index.add_with_ids(encoded_data, np.arange(len(encoded_data)))

In [ ]:
search_text = "harry potter"

In [ ]:
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = 2
distances, ann = index.search(_vector, k=k)
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [ ]:
distances

In [ ]:
xsum_sample["summary"][results['ann'][0]]

In [ ]:
xsum_sample["summary"][results['ann'][1]]

### Summarization

In [19]:
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=60,
    truncation=True,
) 

In [20]:
xsum_sample["summary"][0]

'Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.'

In [21]:
summarizer(xsum_sample["document"][0])

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [21]:
# Apply to a batch of articles
results = summarizer(xsum_sample["document"])

In [22]:
display(
    pd.DataFrame.from_dict(results)
    .rename({"summary_text": "generated_summary"}, axis=1)
    .join(pd.DataFrame.from_dict(xsum_sample))[
        ["generated_summary", "summary", "document"]
    ]
)

,generated_summary,summary,document
0,the full cost of damage in Newton Stewart is s...,Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one..."
1,a fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...
2,Sebastian Vettel will start third ahead of tea...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...
3,the 67-year-old is accused of committing the o...,A former Lincolnshire Police officer carried o...,"John Edward Bates, formerly of Spalding, Linco..."
4,a man receiving psychiatric treatment at the c...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...
5,Gregor Townsend gave a debut to powerhouse win...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,the 25-year-old was hit by a motorbike during ...,Welsh cyclist Luke Rowe says changes to the sp...,Belgian cyclist Demoitie died after a collisio...
8,gundogan will not be fit for the start of the ...,Manchester City midfielder Ilkay Gundogan says...,"Gundogan, 26, told BBC Sport he ""can see the f..."
9,the crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened about 07:20 GMT at the junc...


### Search and sampling in inference

You may see parameters like `num_beams`, `do_sample`, etc. specified in Hugging Face pipelines.  These are inference configurations.

LLMs work by predicting (generating) the next token, then the next, and so on.  The goal is to generate a high probability sequence of tokens, which is essentially a search through the (enormous) space of potential sequences.

To do this search, LLMs use one of two main methods:
* **Search**: Given the tokens generated so far, pick the next most likely token in a "search."
   * **Greedy search** (default): Pick the single next most likely token in a greedy search.
   * **Beam search**: Greedy search can be extended via beam search, which searches down several sequence paths, via the parameter `num_beams`.
* **Sampling**: Given the tokens generated so far, pick the next token by sampling from the predicted distribution of tokens.
   * **Top-K sampling**: The parameter `top_k` modifies sampling by limiting it to the `k` most likely tokens.
   * **Top-p sampling**: The parameter `top_p` modifies sampling by limiting it to the most likely tokens up to probability mass `p`.

You can toggle between search and sampling via parameter `do_sample`.

For more background on search and sampling, see [this Hugging Face blog post](https://huggingface.co/blog/how-to-generate).

We will illustrate these various options below using our summarization pipeline.

In [23]:
# We can instead do a beam search by specifying num_beams.
summarizer(xsum_sample["document"][0], num_beams=10)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]

In [25]:
# Alternatively, we could use sampling.
summarizer(xsum_sample["document"][0], do_sample=True)

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the waters breached a retaining wall, flooding many commercial properties .'}]

In [26]:
# We can modify sampling to be more greedy by limiting sampling to the top_k or top_p most likely next tokens.
summarizer(xsum_sample["document"][0], do_sample=True, top_k=10, top_p=0.8)›

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . the water breached a retaining wall, flooding many commercial properties .'}]